In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, Conv2D, Conv1D, MaxPooling2D, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow import keras, config
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print("GPUs Available: ", len(config.list_physical_devices('GPU')))

In [ ]:
%load_ext autoreload
%autoreload 2
from image_process.image_preprocessors import add_noise, add_noise_randomly, shift_random_update_positions, shift_randomly_position_labels
from data_filters import tempo_interval, take_percent
# Global settings
TEST_SPLIT_SIZE = 0.80
VALIDATION_SPLIT_SIZE = 0.90
IMAGE_TARGET_SIZE = (5, 1400, 1)
BATCH_SIZE = 128
EPOCH_SIZE = 500
LETTER_END_POSITION = "P1"
ADD_NOISE_RANDOMLY = [-2, 30]
ADD_SIGNAL_INDENT_RANDOMLY = [12860, 12860]
SHIFT_MIN_MAX = [-200, -140]
IMAGE_PREPOCESSORS = [
    {"func": shift_randomly_position_labels, "params" : SHIFT_MIN_MAX},
]

In [ ]:
def show_image(img, width, position):
    plt.figure(figsize=(30,5))
    plt.xlim(0, width)
    plt.xticks(position)    
    plt.imshow(img)
    plt.show()

In [ ]:
from Image_Generator_helpers import  DataSets, set_paths, global_path,Random_Item
from data_filters import tempo_interval_raw
set_obj: DataSets = DataSets(set_paths, global_path, [tempo_interval_raw([18, 25])])
set_obj.csv_files

shift_max = abs(SHIFT_MIN_MAX[0] - SHIFT_MIN_MAX[1])
print(shift_max)

# (data_frame["Tempo"].values.astype(np.float) / data_frame["Tempo Diff"].values.astype(np.float)) / 100

def get_position_labels(random_items):
    return [item.csv_row[LETTER_END_POSITION].values.astype(np.float64)[0] for item in random_items]

def position_labels_normalizer(labels, set_obj: DataSets, random_items: list[Random_Item]):
    return np.array([(label - set_obj.min_first_letter_position) / (set_obj.max_first_letter_position + shift_max - set_obj.min_first_letter_position) for label in labels])

def position_labels_de_normalizer(labels, set_obj: DataSets):
    return np.array([(label * (set_obj.max_first_letter_position + shift_max - set_obj.min_first_letter_position) + set_obj.min_first_letter_position) for label in labels])


In [ ]:
from keras_generators.image_generator import Image_Generator_RAW
Train_Generator_RAW = Image_Generator_RAW(
    image_amount=BATCH_SIZE * EPOCH_SIZE,
    set_obj= set_obj,
    FFT_JUMP=64,
    batch_size=BATCH_SIZE,
    image_target_size=IMAGE_TARGET_SIZE,
    image_prepocessors=IMAGE_PREPOCESSORS,
    noise_range=ADD_NOISE_RANDOMLY,
    random_signal_indent=ADD_SIGNAL_INDENT_RANDOMLY,
    label_func = get_position_labels,
    label_post_process=position_labels_normalizer
    )

t, l = Train_Generator_RAW.__getitem__(0)

labels_de_normalized = position_labels_de_normalizer(l, set_obj)

for idx,img in enumerate(t):
    print("label: ")
    pos = labels_de_normalized[idx]
    print(pos)
    show_image(img, 200, [50, 100, 150, 200, pos])

In [ ]:
def conv_model_position(input_layer):
    x = keras.layers.Cropping2D(cropping=((0, 0), (0,1000)), data_format=None)(input_layer)

    x = Conv2D(90,(1,7), padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(2,1),padding="same")(x)

    x = Conv2D(90,(1,7),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Conv2D(90,(1,5),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Conv2D(90,(3,3),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Conv2D(90,(3,3),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Conv2D(90,(3,3),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Conv2D(90,(3,3),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Flatten()(x)

    return x

In [ ]:
from tensorflow.keras import layers

input_layer = Input(shape=IMAGE_TARGET_SIZE)
conv_model_position_flattened = conv_model_position(input_layer)
output_layer_position = Dense(1, name="regr")(conv_model_position_flattened)

model = Model(inputs=input_layer, outputs=output_layer_position)
model.compile(loss=["mse"], optimizer='adam', metrics=["mean_absolute_error"])

print(model.summary())


In [ ]:
init_epoch = 0

In [ ]:

num_epochs = 10

def fit_model(epochs):
	
	global init_epoch
	history = model.fit(
		Train_Generator_RAW,
		steps_per_epoch = EPOCH_SIZE,
		epochs = epochs + init_epoch,
		initial_epoch=init_epoch,
		verbose =1,
		# validation_data = validation_batch_generator,
		# validation_steps = int(len(train_validation) // BATCH_SIZE),
		workers=12,
		use_multiprocessing=True
	)

	
	init_epoch += epochs
	return history

history = fit_model(num_epochs)


In [ ]:
from multiprocessing import Pool

In [ ]:
pixel_difference = 5
def sum_up_to(generator):
    return generator.__getitem__(0)


def get_deviating_predictions(pixel_difference, generator):

    a_pool = Pool()

    result = a_pool.map(sum_up_to, [generator for a in range(generator.__len__())])

    regression_differences = []

    for imgs_batch, labels_batch in result:

        predictions = model.predict_on_batch(imgs_batch) ## make the predictions before the loop, then insert predictions into multiprocessing functions
        predictions = position_labels_de_normalizer(predictions, set_obj)
        labels_batch = position_labels_de_normalizer(labels_batch, set_obj)
        for i in range(len(imgs_batch)):

            regr_pred = predictions[i]
            regr_test_label = labels_batch[i]

            if abs(regr_pred[0] - regr_test_label) > pixel_difference:
                regression_differences.append([regr_pred, regr_test_label, imgs_batch[i]])

    return regression_differences

In [ ]:
%%capture cap --no-stderr
from log_helpers.training_log import Training_Data_Log, print_name, json_to_file
import inspect

data_log = Training_Data_Log()
data_log.model_config = model.to_json()
data_log.model_config_method_string = [inspect.getsource(conv_model_position)]
# data_log.training_sets = get_sets()
# data_log.training_set_size = len(train)
# data_log.validation_set_size = len(train_validation)
# data_log.test_set_size = len(train_test)
data_log.image_pre_processors = print_name(IMAGE_PREPOCESSORS)
data_log.noise_added = ADD_NOISE_RANDOMLY
# data_log.training_data_masks = print_name(MASKS)
data_log.model_summary = model.summary()
data_log.model_optimizer = str(type(model.optimizer))
data_log.model_history = history.history
data_log.model_history_final_epoch = {k: v[-1] for k, v in history.history.items()}
data_log.total_epochs = init_epoch


In [ ]:
class Regression_Results:
    image_preprocessors_test = None
    total_predictions = None
    noise_level = None
    difference_in_pixels = None
    predictions_off_by_more_than_difference = None
    predictions_incorrect_prercent = None
    model_evaluation = None

noise_levels = [0.0]
result_array = []
batches = 200
for noise_level in noise_levels:

    test_batch_generator = Image_Generator_RAW(
        image_amount=BATCH_SIZE * batches,
        set_obj= set_obj,
        FFT_JUMP=64,
        batch_size=BATCH_SIZE,
        image_target_size=IMAGE_TARGET_SIZE,
        image_prepocessors=IMAGE_PREPOCESSORS,
        noise_range=[0, 30],
        random_signal_indent=ADD_SIGNAL_INDENT_RANDOMLY,
        label_func = get_position_labels,
        label_post_process=position_labels_normalizer
    )


    regression_differences = get_deviating_predictions(pixel_difference, test_batch_generator)
    evaluations = model.evaluate(test_batch_generator, verbose = 0)

    results = Regression_Results()
    # results.image_preprocessors_test = print_name(image_preprocessors_test)
    results.total_predictions = batches * BATCH_SIZE
    # results.noise_level = noise_level
    results.difference_in_pixels = pixel_difference
    results.predictions_off_by_more_than_difference = len(regression_differences)
    results.predictions_incorrect_prercent = round( (  len(regression_differences) / (batches * BATCH_SIZE)  ) * 100, 4)
    results.model_evaluation = evaluations

    result_array.append(results.__dict__)

    print(len(regression_differences))


In [ ]:
data_log.results = result_array
print(data_log.results)

In [ ]:
json_to_file("logs/regression/regression_data_log_raw", data_log)

In [ ]:
model.save("saved_model_regresssion_raw_2")

In [ ]:

for idx, diff in enumerate(regression_differences):

    if idx > 100:
        break

    pred, correct, img = diff

    img_pred = img.copy()
    img_correct = img.copy()

    print('Prediction', round(pred[0]))
    img_pred[:, round(int(pred))] = 1
    show_image(img_pred, 500, [round(pred[0])])

    print('Correct', round(correct))
    img_correct[:, round(int(correct))] = 1
    show_image(img_correct, 500, [round(pred[0])])

    print("----------------------------------------------------------------------------------------")
